In [1]:
# System
import os

# Data processing
import numpy as np
import math as m

# Results presentation
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output
from tabulate import tabulate

# NN related stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# from torch.autograd import Variable

import data
import models
import utils
import correlation

%matplotlib inline

In [2]:
class GlobalArguments():
    
    def __init__(self):
        self.model       = 'VGG16BN'
        self.dataset     = 'CIFAR10'
        self.data_path   = 'Data/'
        self.batch_size  = 128
        self.num_workers = 4
        self.transform   = 'VGG'
        self.use_test    = True
        self.models_path = 'Checkpoints/'
        self.n_models    = 17
args = GlobalArguments()

In [3]:
loaders, num_classes = data.loaders(
    args.dataset,
    args.data_path,
    args.batch_size,
    args.num_workers,
    args.transform,
    args.use_test
)

Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


In [4]:
architecture = getattr(models, args.model)

In [5]:
models_list_fge = []
# indicies = range (690, 1011, 20)
indicies = range (210, 1011, 160)

for i in indicies:
    model = architecture.base(num_classes=num_classes, **architecture.kwargs)
    checkpoint = torch.load('./Checkpoints/FGE_VGG16BN_CIFAR10_CYCLE8_3/fge-' + str(i) + '.pt', map_location=torch.device('cpu'))
#     print (model)
    model.load_state_dict(checkpoint['model_state'])
    models_list_fge.append(model)

In [7]:
model1 = models_list_fge[0]
model2 = models_list_fge[5]

In [ ]:
model1 = architecture.base(num_classes=num_classes, **architecture.kwargs)
checkpoint = torch.load('./Checkpoints/FGE_VGG16BN_CIFAR10_CYCLE8_3/fge-' + str(i) + '.pt', map_location=torch.device('cpu'))
model1.load_state_dict(checkpoint['model_state'])

model2 = architecture.base(num_classes=num_classes, **architecture.kwargs)
checkpoint = torch.load('./Checkpoints/FGE_VGG16BN_CIFAR10_CYCLE8_3/fge-' + str(i) + '.pt', map_location=torch.device('cpu'))
model2.load_state_dict(checkpoint['model_state'])

In [8]:
print(tabulate(correlation.layers_diff(model1, model2)))

--  ------  -------  -----  -----
 3  weight     1728  0.141  0.193
 3  bias         64  0.156  0.253
 5  weight    36864  0      0
 5  bias         64  0      0
 8  weight    73728  0.055  0.074
 8  bias        128  0.089  0.117
10  weight   147456  0      0
10  bias        128  0      0
13  weight   294912  0.042  0.055
13  bias        256  0.041  0.052
15  weight   589824  0.034  0.044
15  bias        256  0.058  0.073
17  weight   589824  0      0
17  bias        256  0      0
20  weight  1179648  0.023  0.031
20  bias        512  0.022  0.029
22  weight  2359296  0.014  0.018
22  bias        512  0.047  0.06
24  weight  2359296  0      0
24  bias        512  0      0
27  weight  2359296  0.01   0.013
27  bias        512  0.032  0.042
29  weight  2359296  0.01   0.012
29  bias        512  0.015  0.019
31  weight  2359296  0      0
31  bias        512  0      0
--  ------  -------  -----  -----


In [ ]:
0.append(0)

In [ ]:
# for p in model.modules():
#     print (type(p))
#     for a in p.state_dict().keys():
#         print ('    ', a, p.state_dict()[a].shape)

# for idx, module in enumerate(model.modules()):
#     if type(module) == torch.nn.modules.conv.Conv2d:
#         for key in module.state_dict().keys():
#             print ('[', idx, ']', key, module.state_dict()[key].shape)


In [ ]:
correlation.layers_L2(models_list_fge[0], models_list_fge[1])

In [ ]:
a = torch.nn.parameter.Parameter()
dir(a)

In [ ]:
# a = correlation.correlation_over_classes(model1, model2, loaders['test'], torch.device('cuda:0'))

In [ ]:
correlation.layers_L2(models_list_fge[0], models_list[1], device=torch.device('cuda:0'))

In [ ]:
cor_matrix = correlation.cross_entropy_Nmodels (models_list_fge, loaders['test'], torch.device('cpu'), mean=True)

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar
def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
models_num = [1, 2, 3, 4, 5, 6]
fig, ax = plt.subplots()

im, cbar = heatmap(cor_matrix, models_num, models_num, ax=ax,
                   cmap="YlGn", cbarlabel="corelation")
texts = annotate_heatmap(im, valfmt="{x:.3f}")

fig.tight_layout()
plt.show()

In [ ]:
# cor_matrix = correlation.correlation_among_models (models, loaders['test'], torch.device('cuda:0'), one_hot=False)

In [ ]:
# models_num = [1, 2, 3, 4, 5, 6]
# fig, ax = plt.subplots()

# im, cbar = heatmap(cor_matrix, models_num, models_num, ax=ax,
#                    cmap="YlGn", cbarlabel="corelation")
# texts = annotate_heatmap(im, valfmt="{x:.3f}")

# fig.tight_layout()
# plt.show()